<a href="https://colab.research.google.com/github/NiharikaGopinath/Sentimental-Analysis/blob/main/FinalSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#make sure u have a local file of the dataset on your system

from google.colab import files
uploaded = files.upload()

#creating a pandas dataframe

import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['imdbandsentiment.csv']), encoding="unicode-escape")
#data=pd.read_csv(,encoding="unicode-escape")

#BiLSTM code final
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Flatten,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

#preprocessing of user input

def remove_special_chars(message): 
    for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$","@", "%", "^", "*", "(", ")", "{", "}","[", "]", "|", "/", "\\", ">", "<", "-",
 "!", "?", ".", "'",
 " — ", " — -", "#"]):
        message.replace(remove, "", inplace=True)
        return message


def remove_tags(text):
    return re.compile(r'<[^>]+>').sub('', text)

def remove_num(text):
    return ''.join(re.sub(r"([0–9]+)","",text))
from gensim.parsing.preprocessing import STOPWORDS, remove_stopwords

def remove_stopwords1(text):
    m=text.split()
    m1=[i.lower() for i in m]
    STOPWORDS.union(set(["this","that","the","a","an","there","us"]))
    resultw=[word for word in m1 if word not in STOPWORDS]
    result=' '.join(resultw)
    return result

data.review=data.review.apply(lambda x : x.lower())
data.review=data.review.apply(lambda x : remove_tags(x))
data.review=data.review.apply(lambda x : remove_num(x))

remove_special_chars(data.review)
data.head()

tokenizer = Tokenizer(num_words=5000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
tokenizer.fit_on_texts(data['review'])
data.review=data.review.apply(lambda x : remove_stopwords1(x))
X = tokenizer.texts_to_sequences(data['review'])

X = pad_sequences(X,maxlen=500)
Y = data['label']
vocab_size = len(tokenizer.word_index) + 1
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 24)

from sklearn.preprocessing import LabelEncoder
def targetprep(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc
#BiLSTM Model
ytrain,ytest = targetprep(Y_train,Y_test)
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=500))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history=model.fit(X_train, ytrain,batch_size=128,epochs=20,validation_data=[X_test, ytest], callbacks=[es])

def predict(message):
    remove_stopwords(message)
    x= tokenizer.texts_to_sequences([message])
    x = pad_sequences(x, maxlen=500)
    prediction= model.predict(x)[0][0]
    if prediction > 0.6:
        print("Positive.\nGlad to hear that!")
    elif prediction <0.4:
        print("Negative.\nSorry to hear that")
    else:
        print("I'm not sure what you mean. Please elaborate or rephrase.")



Saving imdbandsentiment.csv to imdbandsentiment.csv
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 50)           6588650   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               183296    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 6,772,203
Trainable params: 6,772,203
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
275/275 [==============================] - 788s 3s/step - loss: 0.5561 - accuracy: 0.7113 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
275/275 [=====================

In [3]:
#The chatbot with basic responses

import random
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey","yo")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
flag=True
print("ASSISSTA: My name is ASSISSTA. I will tell you type of sentiment. If you want to exit, type Bye!")
while(flag==True):
  
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ASSISTA: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ASSISSTA: "+greeting(user_response))
            else:
                print("ASSISSTA: ",end="")
                predict(user_response)
               
    else:
        flag=False
        print("ASSISSTA: Bye! take care..")

ASSISSTA: My name is ASSISSTA. I will tell you type of sentiment. If you want to exit, type Bye!
hey
ASSISSTA: hello
i am very happy today
ASSISSTA: Positive.
Glad to hear that!
i am very sad 
ASSISSTA: Negative.
Sorry to hear that
i hate this product
ASSISSTA: Negative.
Sorry to hear that
glad to experience this
ASSISSTA: Positive.
Glad to hear that!
bye 
ASSISSTA: Positive.
Glad to hear that!
thank you
ASSISTA: You are welcome..
